In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, when

In [2]:
spark = SparkSession \
    .builder \
    .appName("ELT_Client") \
    .config("spark.mongodb.read.connection.uri", "mongodb://127.0.0.1:27017/voiture.client") \
    .config("spark.mongodb.write.connection.uri", "mongodb://127.0.0.1:27017/voiture.client") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.2") \
    .config("spark.sql.catalogImplementation", "hive") \
    .config("spark.sql.hive.metastore.jars", "maven") \
    .config("spark.sql.hive.metastore.version", "3.1.3") \
    .config("spark.sql.hive.metastore.uris", "thrift://127.0.0.1:10000") \
    .enableHiveSupport() \
    .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark-3.5.0-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/vagrant/.ivy2/cache
The jars for the packages stored in: /home/vagrant/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-35f3a1b8-dbac-4490-aa92-bf419b1c9ab6;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.2 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
:: resolution report :: resolve 3075ms :: artifacts dl 59ms
	:: modules in use:
	org.mongodb#bson;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-core;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-sync;4.0.5 from central in [default]
	org.mongodb.spark#mongo-spark-connector_2.12;3.0.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifa

In [3]:
df = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://127.0.0.1:27017/voiture").option("database", "voiture").option("collection", "client").load()

24/05/07 08:32:29 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.rpc.RpcTimeoutException: Futures timed out after [10000 milliseconds]. This timeout is controlled by spark.executor.heartbeatInterval
	at org.apache.spark.rpc.RpcTimeout.org$apache$spark$rpc$RpcTimeout$$createRpcTimeoutException(RpcTimeout.scala:47)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:62)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:58)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:76)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1219)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:295)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java

In [4]:
df = df.withColumn("situationFamiliale", when(df["situationFamiliale"].isin(["Seul", "Seule"]), "Célibataire")
                   .when(df["situationFamiliale"].isin(["?", ""]), "N/D")
                   .otherwise(df["situationFamiliale"]))

In [5]:
df = df.withColumn("sexe", when(df["sexe"].isin(["?", ""]), "N/D")
                   .when(df["sexe"].isin(["Homme", "Masculin"]), "M")
                   .when(df["sexe"].isin(["Femme", "Féminin", "Féminin"]), "F")
                   .otherwise(df["sexe"]))

In [ ]:
df.createOrReplaceTempView("spark_client")

In [6]:
df.write.insertInto('concessionaire.client', overwrite=True)

https://maven-central.storage-download.googleapis.com/maven2/ added as a remote repository with the name: repo-1
Ivy Default Cache set to: /home/vagrant/.ivy2/cache
The jars for the packages stored in: /home/vagrant/.ivy2/jars
org.apache.logging.log4j#log4j-api added as a dependency
org.apache.derby#derby added as a dependency
org.apache.hive#hive-metastore added as a dependency
org.apache.hive#hive-exec added as a dependency
org.apache.hive#hive-common added as a dependency
org.apache.hive#hive-serde added as a dependency
com.google.guava#guava added as a dependency
org.apache.hadoop#hadoop-client-api added as a dependency
org.apache.hadoop#hadoop-client-runtime added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a6da0f5d-6ab4-4890-8540-407ad77a5be2;1.0
	confs: [default]
	found org.apache.logging.log4j#log4j-api;2.10.0 in central
	found org.apache.derby#derby;10.14.1.0 in central
	found org.apache.hive#hive-metastore;3.1.3 in central
	found org.apac

08:37:09.106 [Thread-5] ERROR DataNucleus.Datastore - Exception thrown creating StoreManager. See the nested exception
org.datanucleus.exceptions.NucleusException: Error creating transactional connection factory
	at org.datanucleus.store.AbstractStoreManager.registerConnectionFactory(AbstractStoreManager.java:214) ~[datanucleus-core-4.1.17.jar:?]
	at org.datanucleus.store.AbstractStoreManager.<init>(AbstractStoreManager.java:162) ~[datanucleus-core-4.1.17.jar:?]
	at org.datanucleus.store.rdbms.RDBMSStoreManager.<init>(RDBMSStoreManager.java:285) ~[datanucleus-rdbms-4.1.19.jar:?]
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method) ~[?:1.8.0_402]
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62) ~[?:1.8.0_402]
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45) ~[?:1.8.0_402]
	at java.lang.reflect.Constructor.newInstance(Constructor.java:423) ~[?:1.8.0_402]
	at org.da

AnalysisException: org.apache.hadoop.hive.ql.metadata.HiveException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient

In [ ]:
spark.sql("INSERT INTO TABLE concessionaire.client SELECT * FROM spark_client")